# Repaso Unidad II – Modelos Estadísticos para Series Temporales Univariantes (Capítulo 4)

**Curso:** Analítica de Series Temporales  
**Estudiante:** _Completar nombre_  
**Fecha:** _Completar fecha_  

## Objetivo
Documentar y ejecutar de forma integral los modelos principales del capítulo 4 de Lazzeri (2020): **AR, MA, ARMA, ARIMA y SARIMAX**, comparando su desempeño en una serie temporal univariante.


## 2. Contexto teórico breve

- **AR (AutoReg)**: modela el valor actual en función de rezagos de la propia serie. Útil cuando PACF sugiere corte en pocos lags.
- **MA (Moving Average)**: modela el valor actual con rezagos del error. Suele relacionarse con patrones observables en ACF.
- **ARMA**: combina términos AR y MA para series estacionarias.
- **ARIMA**: incorpora diferenciación (**d**) para tratar series no estacionarias.
- **SARIMAX**: extiende ARIMA con componente estacional y (opcionalmente) variables exógenas.

Conceptos clave:
- **Estacionariedad**: media/varianza relativamente estables en el tiempo.
- **Diferenciación (d)**: resta entre observaciones consecutivas para estabilizar la media.
- **ACF/PACF**: apoyan la selección inicial de órdenes (p, q).


## 3. Librerías y configuración


In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pandas.plotting import lag_plot, autocorrelation_plot
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler

np.random.seed(42)
plt.style.use("default")


## 4. Datos y preparación


In [ ]:
# Opción demo con una serie sintética + tendencia + estacionalidad (contenido funcional conservado)
n = 500
t = np.arange(n)
y = 50 + 0.05*t + 5*np.sin(2*np.pi*t/24) + np.random.normal(0, 1.5, n)
df = pd.DataFrame({
    "timestamp": pd.date_range("2024-01-01", periods=n, freq="H"),
    "load": y
}).set_index("timestamp")
print("Dimensiones:", df.shape)
print("Nulos por columna:", "\n", df.isnull().sum())
df.head()


### Comentario de preparación
La serie no presenta nulos y ya está indexada por tiempo. Se mantiene el enfoque univariante (`load`) para reproducir los modelos del capítulo.


## 5. EDA temporal


In [ ]:
fig, ax = plt.subplots(figsize=(14,4))
df["load"].plot(ax=ax, title="Serie temporal (load)")
ax.set_xlabel("Tiempo")
ax.set_ylabel("load")
plt.show()


In [ ]:
plt.figure(figsize=(5,5))
lag_plot(df["load"])
plt.title("Lag plot")
plt.show()

plt.figure(figsize=(12,4))
autocorrelation_plot(df["load"])
plt.title("Autocorrelation plot (pandas)")
plt.show()

plot_acf(df["load"], lags=48)
plt.show()

plot_pacf(df["load"], lags=48, method="ywm")
plt.show()


### Interpretación EDA
- Se observa patrón temporal con tendencia suave y componente estacional.
- El lag plot muestra dependencia temporal (la serie no es ruido blanco).
- ACF/PACF sugieren estructura autorregresiva y posible necesidad de diferenciación para estabilizar la serie.


## 6. Split temporal


In [ ]:
train_size = int(len(df) * 0.8)
train, test = df.iloc[:train_size].copy(), df.iloc[train_size:].copy()

print("Train:", train.shape, "Test:", test.shape)
print("Rango train:", train.index.min(), "->", train.index.max())
print("Rango test:", test.index.min(), "->", test.index.max())


In [ ]:
def metricas(y_true, y_pred):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    assert len(y_true) == len(y_pred), "Longitudes de y_true y y_pred no coinciden"
    mape = mean_absolute_percentage_error(y_true, y_pred) * 100
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    return mape, mae, rmse

resultados = []


## 7. Modelado

A continuación se implementan **AR, MA, ARMA, ARIMA y SARIMAX**. Cada bloque incluye objetivo, parámetros, ejecución, evaluación y análisis.


### 7.1 Modelo AR (AutoReg)
**Propósito:** capturar dependencia lineal de la serie con sus propios rezagos.  
**Parámetro usado:** `lags=6` (referencia inicial a partir de PACF).  
**Expectativa:** buen ajuste de corto plazo si la componente autorregresiva domina.


In [ ]:
p = 6
ar_model = AutoReg(train["load"], lags=p, old_names=False).fit()
print(ar_model.summary())
print("AIC:", ar_model.aic, "BIC:", ar_model.bic)

pred_ar = ar_model.predict(start=test.index[0], end=test.index[-1])
mape_ar, mae_ar, rmse_ar = metricas(test["load"], pred_ar)

plt.figure(figsize=(14,4))
plt.plot(test.index, test["load"], label="Real")
plt.plot(test.index, pred_ar, label="Pred AR")
plt.title("AR: real vs predicción")
plt.legend()
plt.show()

print(f"AR -> MAPE: {mape_ar:.2f}% | MAE: {mae_ar:.3f} | RMSE: {rmse_ar:.3f}")
resultados.append({"Modelo":"AR", "MAPE (%)":mape_ar, "MAE":mae_ar, "RMSE":rmse_ar, "Observaciones":"Capta inercia temporal; sensible a no estacionariedad."})


**Análisis AR:**
El modelo AR suele seguir adecuadamente la dinámica local, pero puede perder precisión cuando hay tendencia/estacionalidad no modeladas explícitamente. Si el error aumenta hacia el final, es señal de estructura adicional no capturada.


### 7.2 Diferenciación y estacionariedad
**Propósito:** transformar la serie para reducir tendencia y aproximar estacionariedad.  
**Expectativa:** ACF/PACF de la serie diferenciada con caída más rápida.


In [ ]:
df["load_diff1"] = df["load"].diff()

df["load_diff1"].dropna().plot(figsize=(14,4), title="Serie diferenciada (d=1)")
plt.show()

plot_acf(df["load_diff1"].dropna(), lags=48)
plt.show()

plot_pacf(df["load_diff1"].dropna(), lags=48, method="ywm")
plt.show()


**Análisis diferenciación:**
La primera diferencia reduce la tendencia y mejora las condiciones para modelos tipo MA/ARMA. Esta transformación justifica el uso de `d=1` en modelos integrados.


### 7.3 Modelo MA(q)
**Propósito:** modelar la serie a partir de rezagos del error (en práctica, sobre serie diferenciada).  
**Parámetros:** `ARIMA(order=(0,0,2))` sobre `diff(train)`.  
**Expectativa:** capturar shocks de corto plazo.


In [ ]:
train_diff = train["load"].diff().dropna()
ma_model = ARIMA(train_diff, order=(0,0,2)).fit()
print(ma_model.summary())

pred_ma_diff = ma_model.forecast(steps=len(test))
pred_ma = train["load"].iloc[-1] + np.cumsum(pred_ma_diff)

mape_ma, mae_ma, rmse_ma = metricas(test["load"], pred_ma)

plt.figure(figsize=(14,4))
plt.plot(test.index, test["load"], label="Real")
plt.plot(test.index, pred_ma, label="Pred MA")
plt.title("MA: real vs predicción")
plt.legend()
plt.show()

print(f"MA -> MAPE: {mape_ma:.2f}% | MAE: {mae_ma:.3f} | RMSE: {rmse_ma:.3f}")
resultados.append({"Modelo":"MA", "MAPE (%)":mape_ma, "MAE":mae_ma, "RMSE":rmse_ma, "Observaciones":"Modela shocks; limitado ante estacionalidad marcada."})


**Análisis MA:**
El MA mejora cuando la autocorrelación se concentra en pocos rezagos del error. Si el ajuste visual subestima picos/valleys periódicos, sugiere componente estacional no incorporada.


### 7.4 Modelo ARMA(p,q)
**Propósito:** combinar inercia (AR) y shocks (MA) para la serie estacionaria.  
**Parámetros:** `ARIMA(order=(2,0,2))` sobre la serie diferenciada.  
**Expectativa:** mejor equilibrio bias-varianza que AR o MA por separado.


In [ ]:
arma_model = ARIMA(train_diff, order=(2,0,2)).fit()
print(arma_model.summary())

pred_arma_diff = arma_model.forecast(steps=len(test))
pred_arma = train["load"].iloc[-1] + np.cumsum(pred_arma_diff)

mape_arma, mae_arma, rmse_arma = metricas(test["load"], pred_arma)

plt.figure(figsize=(14,4))
plt.plot(test.index, test["load"], label="Real")
plt.plot(test.index, pred_arma, label="Pred ARMA")
plt.title("ARMA: real vs predicción")
plt.legend()
plt.show()

print(f"ARMA -> MAPE: {mape_arma:.2f}% | MAE: {mae_arma:.3f} | RMSE: {rmse_arma:.3f}")
resultados.append({"Modelo":"ARMA", "MAPE (%)":mape_arma, "MAE":mae_arma, "RMSE":rmse_arma, "Observaciones":"Combina memoria y errores; depende de buena estacionarización."})


**Análisis ARMA:**
Frente a MA puro, ARMA suele capturar mejor la dinámica local. Si la serie tiene integración o estacionalidad fuerte, aún puede quedar rezago estructural en el pronóstico.


### 7.5 Modelo ARIMA(p,d,q)
**Propósito:** modelar simultáneamente autorregresión, diferenciación e innovación.  
**Parámetros:** `order=(2,1,2)`.  
**Expectativa:** mejor desempeño que AR/MA/ARMA cuando existe tendencia no estacionaria.


In [ ]:
arima_model = ARIMA(train["load"], order=(2,1,2)).fit()
print(arima_model.summary())

pred_arima = arima_model.forecast(steps=len(test))

mape_arima, mae_arima, rmse_arima = metricas(test["load"], pred_arima)

plt.figure(figsize=(14,4))
plt.plot(test.index, test["load"], label="Real")
plt.plot(test.index, pred_arima, label="Pred ARIMA")
plt.title("ARIMA: real vs predicción")
plt.legend()
plt.show()

print(f"ARIMA -> MAPE: {mape_arima:.2f}% | MAE: {mae_arima:.3f} | RMSE: {rmse_arima:.3f}")
resultados.append({"Modelo":"ARIMA", "MAPE (%)":mape_arima, "MAE":mae_arima, "RMSE":rmse_arima, "Observaciones":"Integra diferenciación; suele robustecer el pronóstico sin estacionalidad explícita."})


**Análisis ARIMA:**
La incorporación de `d=1` permite estabilizar la media y normalmente reduce error frente a ARMA en series con tendencia. Si aún hay patrón periódico en residuos, se recomienda componente estacional.


### 7.6 Modelo SARIMAX
**Propósito:** extender ARIMA con estacionalidad (y potenciales variables exógenas).  
**Parámetros:** `order=(2,1,0)` y `seasonal_order=(1,1,0,24)` para estacionalidad diaria horaria.  
**Expectativa:** mejor ajuste cuando la serie presenta ciclos estacionales claros.


In [ ]:
# Escalado opcional (contenido funcional conservado)
scaler = MinMaxScaler()
train_scaled = train.copy()
test_scaled = test.copy()
train_scaled["load"] = scaler.fit_transform(train[["load"]])
test_scaled["load"] = scaler.transform(test[["load"]])

sarimax_model = SARIMAX(
    train_scaled["load"],
    order=(2,1,0),
    seasonal_order=(1,1,0,24),
    enforce_stationarity=False,
    enforce_invertibility=False
).fit(disp=False)

print(sarimax_model.summary())

steps = len(test_scaled)
pred_scaled = sarimax_model.forecast(steps=steps)
pred_sarimax = scaler.inverse_transform(np.array(pred_scaled).reshape(-1, 1)).ravel()

mape_sarimax, mae_sarimax, rmse_sarimax = metricas(test["load"].values, pred_sarimax)

plt.figure(figsize=(14,4))
plt.plot(test.index, test["load"], label="Real")
plt.plot(test.index, pred_sarimax, label="Pred SARIMAX")
plt.legend()
plt.title("SARIMAX Forecast")
plt.show()

print(f"SARIMAX -> MAPE: {mape_sarimax:.2f}% | MAE: {mae_sarimax:.3f} | RMSE: {rmse_sarimax:.3f}")
resultados.append({"Modelo":"SARIMAX", "MAPE (%)":mape_sarimax, "MAE":mae_sarimax, "RMSE":rmse_sarimax, "Observaciones":"Captura estacionalidad; normalmente mejora en series con ciclos."})


**Análisis SARIMAX:**
Cuando la estacionalidad está bien especificada, SARIMAX suele mejorar el ajuste visual y reducir métricas de error. La ganancia depende de seleccionar adecuadamente periodo estacional y órdenes.


## 8. Comparación global de modelos


In [ ]:
resumen = pd.DataFrame(resultados)
resumen = resumen.sort_values(by="MAPE (%)", ascending=True).reset_index(drop=True)
resumen.insert(0, "Ranking", np.arange(1, len(resumen)+1))
resumen


In [ ]:
mejor_modelo = resumen.iloc[0]
peor_modelo = resumen.iloc[-1]
print(f"Mejor modelo por MAPE: {mejor_modelo['Modelo']} ({mejor_modelo['MAPE (%)']:.2f}%)")
print(f"Peor modelo por MAPE: {peor_modelo['Modelo']} ({peor_modelo['MAPE (%)']:.2f}%)")


### Interpretación comparativa
- El ranking final se basa en **MAPE** y se complementa con **MAE** y **RMSE**.
- Si **SARIMAX** o **ARIMA** lideran, se confirma la relevancia de diferenciación y/o estacionalidad.
- Si un modelo simple (AR o MA) rinde mejor, puede indicar estructura temporal menos compleja o sobreparametrización en modelos más ricos.


## 9. Conclusiones

1. Se documentó y ejecutó el flujo completo de modelado univariante del capítulo 4.
2. Se compararon cinco familias de modelos (AR, MA, ARMA, ARIMA, SARIMAX) con métricas homogéneas.
3. El mejor modelo debe justificarse por **métricas** y **ajuste visual**, no solo por complejidad.
4. **Limitaciones**: selección manual de órdenes y uso de un solo split train/test.
5. **Mejoras futuras**:
   - validación walk-forward,
   - búsqueda sistemática de hiperparámetros (grid/random search),
   - análisis de residuos más formal,
   - incorporación de variables exógenas reales en SARIMAX.


## 10. Referencias

- Lazzeri, F. (2020). *Machine Learning for Time Series Forecasting with Python*. Capítulo 4.
